# Long candle strategy  EDA

Long candle strategy is classification model.
Features: prices
Targets: -1,0,1 signal

To install conda and binance packages to this notebook uncomment the code below

In [1]:
%%capture
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
%pip install pandas
#%pip install huobi-sdk==2.3.3

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [43]:
#mode="dev" # dev - work with local dev data
mode="yc" # yc - work with data from yandex cloud bots

symbol="BTC-USDT"
strategy="LongCandleDenseStrategy"

In [4]:

import os

# Downloading latest pytrade2 data
if mode == "yc":
    os.system("cd ./../deploy/yandex_cloud; ./download_data.sh")
    data_dir=f"../data/yandex-cloud"
else:
    data_dir=f"../data/dev"
    
print(f"Download completed. Local data dir: {data_dir}")

    

deploy_lib.sh: line 4: yc: command not found
INFO: Retrieving list of remote files for s3://pytrade2/data/ ...


Sync pytrade2 data from s3://pytrade2/data to /home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud


INFO: No cache file found, creating it.
INFO: Compiling list of local files...
INFO: Running stat() and reading/calculating MD5 values on 1697 files, this may take some time...
INFO: [1000/1697]
INFO: Found 24 remote files, 1697 local files
INFO: Verifying attributes...
INFO: disabled md5 check for LongCandleDenseStrategy/Xy/2023-10-07_BTC-USDT_x.csv
INFO: disabled md5 check for LongCandleDenseStrategy/Xy/2023-10-08_BTC-USDT_x.csv
INFO: disabled md5 check for LongCandleDenseStrategy/Xy/2023-10-09_BTC-USDT_x.csv
INFO: Summary: 4 remote files to download, 1675 local files to delete, 0 local files to hardlink


download: 's3://pytrade2/data/LongCandleDenseStrategy/Xy/2023-10-13_BTC-USDT_learn_x.csv' -> '/home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud/LongCandleDenseStrategy/Xy/2023-10-13_BTC-USDT_learn_x.csv'  [1 of 4]
 194681 of 194681   100% in    0s  1102.88 kB/s  done
download: 's3://pytrade2/data/LongCandleDenseStrategy/Xy/2023-10-13_BTC-USDT_learn_y.csv' -> '/home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud/LongCandleDenseStrategy/Xy/2023-10-13_BTC-USDT_learn_y.csv'  [2 of 4]
 7094 of 7094   100% in    0s   142.22 kB/s  done
download: 's3://pytrade2/data/LongCandleDenseStrategy/Xy/2023-10-12_BTC-USDT_learn_x.csv' -> '/home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud/LongCandleDenseStrategy/Xy/2023-10-12_BTC-USDT_learn_x.csv'  [3 of 4]
 1043985 of 1043985   100% in    0s     3.72 MB/s  done
download: 's3://pytrade2/data/LongCandleDenseStrategy/Xy/2023-10-12_BTC-USDT_learn_y.csv' -> '/home/dima/projects/pytrade2/deploy

## Predicted and actual low/high

In [54]:
import os
import glob
import pandas as pd
from datetime import timedelta,date
import matplotlib.pyplot as plt
import plotly.express as px

def read_last_data(data_dir, strategy):
    """ Read days from start to end from local predictions history"""
    
    def read_last(symbol: str, name: str, n: int):
        xy_dir=f"{data_dir}/{strategy}/Xy"
        
        file_path = sorted([f for f in os.listdir(xy_dir) if f.endswith(f"{symbol}_{name}.csv")])[-1]
        file_path = f"{xy_dir}/{file_path}"
        print(f"Read from {file_path}")
        return pd.read_csv(file_path, parse_dates=True).tail(n)
    n=1000
    #data=read_last(symbol, "data", n)
    X=read_last(symbol, "x", n)
    y=read_last(symbol, "y", n)
    data=pd.merge(X,y, left_index=True, right_index=True) 
    #y=pd.concat([data,y])
    return data,X,y


# Read candles for last 3 days
print(symbol)
data,x,y = read_last_data(data_dir, strategy)
y.tail()

BTC-USDT
Read from ../data/yandex-cloud/LongCandleDenseStrategy/Xy/2023-10-11_BTC-USDT_x.csv
Read from ../data/yandex-cloud/LongCandleDenseStrategy/Xy/2023-10-11_BTC-USDT_y.csv


,Unnamed: 0,signal_pred,signal_target
6525,2023-10-11 09:16:12.430,0,0
6526,2023-10-11 09:16:13.973,0,0
6527,2023-10-11 09:16:16.132,0,0
6528,2023-10-11 09:16:16.791,0,0
6529,2023-10-11 09:16:18.154,0,0


In [55]:
y[y['signal_target']!=0].tail()

,Unnamed: 0,signal_pred,signal_target
6337,2023-10-11 09:07:02.841,0,-1
6338,2023-10-11 09:07:03.355,0,-1
6339,2023-10-11 09:07:04.973,0,-1
6340,2023-10-11 09:07:06.248,0,-1
6341,2023-10-11 09:07:08.087,0,-1
